## Create Local DuckDB Database

This notebook consolidates all the generated CSV data into a single, efficient DuckDB database file (`financial_data.duckdb`). DuckDB is a fast, in-process analytical database that is perfect for this kind of local data work—no server required.

This script will:
1.  Scan the data directories for all the price, market cap, and financial statement CSV chunks.
2.  Combine the chunks for each data type into a single table.
3.  Load the two main symbol lists.
4.  Create a new DuckDB database file in the `Analysis` directory with seven tables:
    -   `prices`
    -   `market_caps`
    -   `prices_with_market_cap`
    -   `statements`
    -   `universal_symbols`
    -   `fetchable_symbols`
    -   `company_profiles`

Once this notebook is run, you can connect to the `financial_data.duckdb` file from any other script or notebook to run fast SQL queries on your entire dataset.


In [20]:
import duckdb
import pandas as pd
import os
import glob
import requests

# --- Configuration ---
DB_FILE = 'financial_data.duckdb'

DATA_PATHS = {
    'prices': '../Prices/Price Data/price_data_*.csv',
    'market_caps': '../Market Caps/Market Cap Data/market_cap_data_*.csv',
    'statements': '../Statements/Statement Data/financials_*.csv',
    'universal_symbols': '../Ticker Symbols/Symbol Lists/universal_symbols.csv',
    'fetchable_symbols': '../Ticker Symbols/Symbol Lists/fetchable_symbols.csv',
    'company_profiles': '../Ticker Symbols/Symbol Lists/company_profiles.csv'
}
SYMBOL_CHANGE_URL = 'https://financialmodelingprep.com/stable/symbol-change?invalid=false&limit=20000&apikey=REDACTED'

# --- Database Creation ---

# Remove the old DB file if it exists to start fresh
if os.path.exists(DB_FILE):
    os.remove(DB_FILE)
    print(f"Removed old database file: {DB_FILE}")

# Connect to DuckDB. It will create the file if it doesn't exist.
con = duckdb.connect(database=DB_FILE, read_only=False)
print(f"Created and connected to new DuckDB database: {DB_FILE}")

# --- Table Loading ---

def load_chunked_data(table_name, path_glob):
    """Finds CSVs, combines them, and loads them into a DuckDB table."""
    print(f"--- Processing table: {table_name} ---")
    files = glob.glob(path_glob)
    if not files:
        print(f"Warning: No files found for glob: {path_glob}")
        return
        
    print(f"Found {len(files)} files to combine for '{table_name}'.")
    
    # Use DuckDB's powerful CSV reader to handle the concatenation and table creation
    # This is much more memory-efficient than loading into pandas first.
    con.execute(f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM read_csv_auto('{path_glob}')")
    
    print(f"Successfully created and loaded table '{table_name}'.")

def load_single_file(table_name, path):
    """Loads a single CSV file into a DuckDB table."""
    print(f"--- Processing table: {table_name} ---")
    if not os.path.exists(path):
        print(f"Warning: File not found: {path}")
        return
    
    con.execute(f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM read_csv_auto('{path}')")
    print(f"Successfully created and loaded table '{table_name}'.")

def load_symbol_changes(table_name, url):
    """Fetches symbol change data from the API and loads it into a DuckDB table."""
    print(f"--- Processing table: {table_name} ---")
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        if not data:
            print("Warning: No data received from symbol change API.")
            return
            
        df = pd.DataFrame(data)
        df = df[['date', 'oldSymbol', 'newSymbol']] # Ensure correct columns and order
        
        # Use DuckDB's `from_df` to load the DataFrame into a table
        con.execute(f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM df")
        print(f"Successfully created and loaded table '{table_name}'.")
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching symbol change data: {e}")


# Load the chunked data
load_chunked_data('prices', DATA_PATHS['prices'])
load_chunked_data('market_caps', DATA_PATHS['market_caps'])
load_chunked_data('statements', DATA_PATHS['statements'])

# Load the single-file symbol lists
load_single_file('universal_symbols', DATA_PATHS['universal_symbols'])
load_single_file('fetchable_symbols', DATA_PATHS['fetchable_symbols'])
load_single_file('company_profiles', DATA_PATHS['company_profiles'])

# Load the symbol change data from the API
load_symbol_changes('symbol_changes', SYMBOL_CHANGE_URL)

# --- Create Joined Table ---
print("\n--- Creating joined prices and market cap table ---")
join_query = """
CREATE TABLE prices_with_market_cap AS
SELECT
    p.date,
    p.symbol,
    p.adjClose,
    m.marketCap
FROM prices p
inner JOIN market_caps m ON p.symbol = m.symbol AND p.date = m.date
where marketCap is not null and adjClose is not null;
"""
con.execute(join_query)
print("Successfully created 'prices_with_market_cap' table.")


# --- Verification ---
print("\n--- Verifying Database Contents ---")
tables = con.execute("SHOW TABLES").df()
print("Tables in database:")
print(tables)

print("\nRow counts for each table:")
for table_name in tables['name']:
    count = con.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
    print(f"- {table_name}: {count:,} rows")

# Close the connection
con.close()

print("\nDatabase creation complete.")



Removed old database file: financial_data.duckdb
Created and connected to new DuckDB database: financial_data.duckdb
--- Processing table: prices ---
Found 4 files to combine for 'prices'.
Successfully created and loaded table 'prices'.
--- Processing table: market_caps ---
Found 4 files to combine for 'market_caps'.
Successfully created and loaded table 'market_caps'.
--- Processing table: statements ---
Found 4 files to combine for 'statements'.
Successfully created and loaded table 'statements'.
--- Processing table: universal_symbols ---
Successfully created and loaded table 'universal_symbols'.
--- Processing table: fetchable_symbols ---
Successfully created and loaded table 'fetchable_symbols'.
--- Processing table: company_profiles ---
Successfully created and loaded table 'company_profiles'.
--- Processing table: symbol_changes ---
Successfully created and loaded table 'symbol_changes'.

--- Creating joined prices and market cap table ---


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Successfully created 'prices_with_market_cap' table.

--- Verifying Database Contents ---
Tables in database:
                     name
0        company_profiles
1       fetchable_symbols
2             market_caps
3                  prices
4  prices_with_market_cap
5              statements
6          symbol_changes
7       universal_symbols

Row counts for each table:
- company_profiles: 36,033 rows
- fetchable_symbols: 12,800 rows
- market_caps: 26,275,891 rows
- prices: 28,853,829 rows
- prices_with_market_cap: 26,113,898 rows
- statements: 603,196 rows
- symbol_changes: 4,939 rows
- universal_symbols: 85,850 rows

Database creation complete.


## How to Query the Database

Now that the `financial_data.duckdb` file exists, you can connect to it from any Python script or Jupyter notebook to run SQL queries.

-   **Connect in `read_only` mode**: It's a good practice to connect in read-only mode for analysis to prevent accidentally modifying the data.
-   **Use `.df()`**: The `.df()` method on a query result will return the data directly into a pandas DataFrame, which is extremely convenient for analysis and visualization.

Below is a simple example of how to connect and run a query.


In [30]:
import duckdb
import os

DB_FILE = 'financial_data.duckdb'

if not os.path.exists(DB_FILE):
    print(f"Database file not found at '{DB_FILE}'. Please run the creation script first.")
else:
    # Connect to the existing database in read-only mode
    con = duckdb.connect(database=DB_FILE, read_only=True)
    
    print("--- Successfully connected to the database. ---")
    
    # Example Query: Get the most recent market cap for a few specific stocks
    query = """
    SELECT 
        count(distinct symbol)
        ,count(distinct case when marketCap > 100000000 then symbol end) as "100M"
        ,count(distinct case when marketCap > 1000000000 then symbol end) as "1B"
    FROM prices_with_market_cap
    ;
    """
    
    result_df = con.execute(query).df()
    print(result_df)
    
    # Always remember to close the connection
    con.close()



--- Successfully connected to the database. ---
   count(DISTINCT symbol)  100M    1B
0                   10532  8932  5505
